In [143]:
import unicodedata

def remove_accents(row):
    nfkd_form = unicodedata.normalize('NFKD', row).lower()
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [144]:
import gzip, pickle

with gzip.open('dicionario_wb.dict.gz','rb') as fp:
    dicionario = pickle.load(fp)
    fp.close()
    
with gzip.open('lista_nomes_conjugado.dict.gz','rb') as fp:
    lista_nomes_conjugado = pickle.load(fp)
    fp.close()

In [145]:
def porcentagem(parte,total):
        #parte = contador_nomes_retirados
        #total = contador_nomes_HAREM
        porcentagem = None
        if total != 0:  
            porcentagem = (parte / total)*100
        
        print("Porcentagem: ", porcentagem)       

In [146]:
def porcentagem_parc(parte,total):
    porcentagem_parc = None
    if total != 0:
        porcentagem_parc = (parte/total)*100
        
    print("Porcentagem Parcial: ",porcentagem_parc)    

In [210]:
import re
import xml.etree.ElementTree as ET

def remove_names(row, filename):
    a = os.getcwd()+'/HAREM_EVO/HAREMRELREL.xml'
    #f = open(a)
    #golden_lines = f.readlines()
    tree = ET.parse(a)
    palavras_removidas = []
    root = tree.getroot()
    #for child in root:
    #    print(child.get('DOCID'))
    
    ## Contadores
    contador_nomes_HAREM = 0
    contador_nomes_retirados = 0
    contador_nomes_parcialmente_retirados = 0    
    
    
    ## Separa cada uma das palavras
    sentencas = row.split('\n')
    #print(sentencas)
    j = 0
    for s in sentencas:
        palavras = s.split(' ')
        ## Percorre todas as palavras
        for i, p in enumerate(palavras):           

            ## Remove acentos
            ## Coloca palavras em minúsculas 
            ## Substitue caracteres especiais por espaço
            p = remove_accents(p).lower()
            n = re.sub(r'\W',' ',p).strip()

            #for palavra in dicionario:

                # verifica se no inicio ou no fim há um nome
                # remove do dicionario a palavra toda se houver 
                #if parte.rfind(nome) == (len(parte) - len(nome)): *** TEM QUE VERIFICAR A PALAVRAAAAAA
                    #print ('NomeDic:', nome )
                    #remove(parte.rfind(nome))    QUERO QUE REMOVA A PALAVRA DO DICIONARIO, NAO O NOME DELA!
        


            ## Substitue Nomes
            if n in lista_nomes_conjugado:
                #print ('nomes > ',n)
                print("---Nome Removido---")
                print(n, p)
                palavras[i] = p.lower().replace(n,'NoInfo')
                palavras_removidas.append(n)
            
            else:
                for parte in n.split(' '):

                    #print(parte)
                    ### Substitue nomes no meio de palavras (que não existem no dicionário)
                    if not remove_accents(parte) in dicionario:
                        #if len(parte) > 3:
                            #print('No Dict: ',parte)
                        for nome in lista_nomes_conjugado:
                            in_list = False
                            if parte == nome: #NÃO PEGA O NOME INTEIRO CORRETAMENTE D:
                                p = p.replace(nome,'NoInfo')
                                #print('Inteiro', parte,'*', nome)
                                in_list = True
                                palavras_removidas.append(nome)
                        for nome in lista_nomes_conjugado:
                            ## Remove nomes no meio da palavra
                            #if len(nome) > 3 and parte.find(nome) >= 0:
                            #    print('find:',parte,'|',nome)
                            ## Remove nomes no início da palavra
                            if len(nome) > 3 and parte.find(nome) == 0: 
                                #print('Inicio:',parte,'|',nome)
                                if not in_list:
                                    in_list = True
                                    palavras_removidas.append(nome)
                                p = p.replace(nome,'NoInfo ')
                            ## Remove nomes no final da palavra
                            if len(nome) > 3 and parte.rfind(nome) > 0 and parte.rfind(nome) == (len(parte)-len(nome)):
                                #print('Final:', parte,'-',nome)
                                if not in_list:
                                    in_list = True
                                    palavras_removidas.append(nome)
                                p = p.replace(nome,' NoInfo ')

                        palavras[i] = p.replace(nome,' NoInfo ')


            ## Substitue os números de registros
            num = re.findall(r'\d*',p)[0]
            if len(num) > 7:
                #print (p)
                palavras[i] = 'NoInfo'

            ## Substitue o Leito
            if len(num) >= 3 and palavras[i-1].lower() == 'leito:':
                #print(p, palavras[i-1])
                palavras[i] = 'NoInfo'
        s = ' '.join(palavras)
        
        prev_find = s.find('NoInfo')

        
    #while prev_find != -1:
    #Terminar de comparar os nomes retirados dos textos com os do HArem
    palavras_removidas = set(palavras_removidas)
    print('---- Nomes Removidos ----')
    print(palavras_removidas)
    for doc in root:
        #print("Filename", filename)
        if doc.get('DOCID') == filename:
            #print("DOCID", filename)
            for paragraph in doc:
                for em in paragraph:
                    if em.get("CATEG") == "PESSOA" and em.get('TIPO') == "INDIVIDUAL":
                        #remover acentos

                        em_temp = re.sub(r'\W',' ',em.text).strip()
                        
                        nomes = em_temp.split()
                        
                        # lista de nomes sem acento
                        for nome in nomes:

                            #p = remove_accents(p).lower()

                            nome_sem_acento = remove_accents(nome).lower()

                            contador_nomes_HAREM += len(nomes)

                            

                            #for nome in nomes:
                            print("-----Comparando Nomes----")
                            print(nome)
                            #print(palavras_removidas)
                            for palavra_removida in palavras_removidas:
                                if nome_sem_acento == palavra_removida.lower():
                                    contador_nomes_retirados += 1
                                    print(nome_sem_acento, " |t ", palavra_removida)
                                elif nome_sem_acento.find(palavra_removida.lower()) != -1:
                                    contador_nomes_parcialmente_retirados += 1
                                    print(nome_sem_acento, " |p ", palavra_removida)
            break
                                                    
            #for gl in golden_lines:
            #    if gl.find('DOCID='):
            #        doc_id = gl[gl.find('"')+1:gl.findr('"')]
            #        print(doc_id)
            #        if doc_id == filename:
                        
            #E aí comparar com cada palavras retirada em palavras_removidas para ver se está correta.
            prev_find = s.find('NoInfo', prev_find+1)
            j = j + 1
        #break
    
    
    print("Nomes no HAREM: ", contador_nomes_HAREM, " | Nomes Retirados: ", contador_nomes_retirados, " | Nomes Parcialmente Retirados: ", contador_nomes_parcialmente_retirados, " |Número de Erros: ", contador_nomes_HAREM - contador_nomes_parcialmente_retirados - contador_nomes_retirados)     
    
    porcentagem(contador_nomes_retirados, contador_nomes_HAREM)
    porcentagem_parc(contador_nomes_parcialmente_retirados, contador_nomes_HAREM)
    
    ## Une todas as palavras de volta
    return ' '.join(palavras)

In [211]:
import os, pickle
import xml.etree.ElementTree as ET

a = os.getcwd()+'/HAREM_EVO/HAREMRELREL.xml'
#f = open(a)
#golden_lines = f.readlines()
tree = ET.parse(a)
palavras_removidas = []
root = tree.getroot()
    
c = os.listdir(os.getcwd()+'/HAREM_EVO/')
a = os.getcwd()+'/HAREM_EVO/'
conteudo = []
achou = False
for p in c:
    if p.endswith(".txt"):
        file = open(a+p)
        #print(p)
        for doc in root:
            if doc.get('DOCID') == p.replace('.txt', ''):
                #print (doc)
                achou = True
                remove_names(file.read(), p.replace('.txt', ''))
                break
        #print(file.read())"/textosnerp/"
        #conteudo.append(file.read())
        file.close()
    
    if achou: break

#print(conteudo)

---Nome Removido---
vasconcelos vasconcelos,
---Nome Removido---
policial policial
---Nome Removido---
soraia soraia
---Nome Removido---
chaves chaves
---Nome Removido---
nicolau nicolau
---Nome Removido---
ivo ivo
---Nome Removido---
jose jose
---Nome Removido---
alicia alicia
---Nome Removido---
joaquim joaquim
---Nome Removido---
almeida almeida.
---Nome Removido---
vasconcelos vasconcelos,
---Nome Removido---
tiago tiago
---Nome Removido---
santos santos,
---Nome Removido---
jose jose
---Nome Removido---
antonio antonio
---Nome Removido---
sofia sofia
---Nome Removido---
jose jose
---Nome Removido---
eduardo eduardo.
---Nome Removido---
vasconcelos vasconcelos,
---- Nomes Removidos ----
{'vasconcelos', 'jose', 'almeida', 'sofia', 'eduardo', 'alicia', 'joaquim', 'pedro', 'soraia', 'santos', 'antonio', 'nicolau', 'ivo', 'tiago', 'chaves', 'policial'}
-----Comparando Nomes----
António
antonio  |t  antonio
-----Comparando Nomes----
Pedro
pedro  |t  pedro
-----Comparando Nomes----
Vasco

In [ ]:
# Antonio-Pedro tá pegando pedro e Antonio não. Tem que tirar o - e espaçar onde era o -.

In [ ]:
#evo = pd.read('nerp-crf.tar.gz')

#evo.to_csv('nerp-crf.tar.gz', compression='gzip')

In [ ]:
#print(conteudo[0])
#for c in conteudo:
#    print(remove_names(c))

In [ ]:
for c in conteudo:
    print(c)
    print('\n')

### Porcentagem de Acerto

In [ ]:
# Achar onde noinfo tem '<prop>' do lado

# Apos os acréscimos de noInfo correto, incorreto e dos não identificados, pegar a soma de todos e fazer regra de 3
# ou dividir o correto/total, incorreto/total e o naoID/total e contar como porcentagem, para cada texto e evolucao.

#porcentagem = 100*(part, total);
#    return 100*flutuador(part)/float(total)

In [ ]:
a = os.getcwd()+'/HAREM_EVO/HAREMNAOANOTADO.xml'
tree = ET.parse(a)
root = tree.getroot()
for doc in root:
    filename = doc.get('DOCID')
    f = open(os.getcwd()+'/HAREM_EVO/'+filename+".txt", 'w')
    for paragraph in doc:
        f.write(paragraph.text.strip('\n'))
        f.write('\n')
    f.close()